In [39]:
text_file = sc.textFile("spark.txt")
data = text_file.flatMap(lambda l : l.split(" "))\
                .map(lambda w : (w,1))\
                .reduceByKey(lambda a,b:a+b)\
                .filter(lambda x : x[1] >= 2)

data.collect()

[('Spark', 4), ('is', 3), ('a', 2), ('This', 2)]

In [40]:
data = data.map(lambda p : (p[1],p[0]))\
           .sortByKey(ascending=True)
data.collect()

[(2, 'a'), (2, 'This'), (3, 'is'), (4, 'Spark')]

In [46]:
mapLink = sc.parallelize([["PageA","PageB"],["PageA","PageC"],["PageB","PageC"],["PageC","PageA"],["PageD","PageC"]])

links = mapLink.groupByKey()
print(list((k,list(v)) for (k,v) in links.collect()))
ranks = links.map(lambda pairs : (pairs[0],1))
print(ranks.collect())

[('PageA', ['PageB', 'PageC']), ('PageB', ['PageC']), ('PageD', ['PageC']), ('PageC', ['PageA'])]
[('PageA', 1), ('PageB', 1), ('PageD', 1), ('PageC', 1)]


In [49]:
def computeContribs(urls, rank):
    """Calculates Url contributions to the rank ofother URLs"""
    num_urls = len(urls)
    for url in urls:
        yield (url, rank/ num_urls)

contribs = links.join(ranks).flatMap(lambda url_urls_rank : computeContribs(url_urls_rank[1][0],url_urls_rank[1][1]))
contribs.collect()

[('PageB', 0.5),
 ('PageC', 0.5),
 ('PageC', 1.0),
 ('PageC', 1.0),
 ('PageA', 1.0)]

In [51]:
new_rank = contribs.reduceByKey(lambda x,y:x+y)\
                    .mapValues(lambda rank:0.15+0.85*rank).collect()
print(new_rank)

[('PageB', 0.575), ('PageA', 1.0), ('PageC', 2.275)]
